<a href="https://colab.research.google.com/github/nkcong206/Travel-Recommendation-System/blob/main/crawl_data/crawl_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain-ollama
!pip install -q langchain-google-genai
!pip install -q geopy
!pip install -q selenium
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00


In [44]:
import requests
from bs4 import BeautifulSoup as bp
import time
import csv
import json
import psycopg2
from geopy.geocoders import Nominatim
import re
from langchain_community.chat_models import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from sentence_transformers import SentenceTransformer
from shapely.geometry import Point

In [63]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

Enter your postgresql url: ··········


In [10]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'en-US, en;q=0.5',
}

In [31]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
def get_embedding(text):
    """Trả về embedding của văn bản sử dụng mô hình SentenceTransformer."""
    embedding = model.encode(text)
    return embedding.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

### Gemini

In [6]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    )

### Llama3

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 12030    0 12030    0     0  43071      0 --:--:-- --:--:-- --:--:-- 42964
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [ ]:
#!ollama serve&

In [ ]:
!ollama pull llama3:latest

[GIN] 2024/08/18 - 05:22:56 | 200 |      29.284µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  29 KB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 8.5 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  31 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  62 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  70 MB/4.7 GB                  pul

In [ ]:
llm = ChatOllama(model="llama3:latest", temperature=0)

### Hotel

In [61]:
def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text

def get_hotel_price(soup):
    price_element = soup.find('div', attrs={'style': 'color: rgb(255, 94, 31); font-size: 20px;'})
    if price_element:
        true_price = price_element.text.strip().replace('VND', '').replace('.', '')
        return float(true_price)
    return None

def get_hotel_name(soup):
    name = soup.find('div', class_='css-901oao r-a5wbuh r-1enofrn r-b88u0q r-1cwl3u0 r-fdjqy7 r-3s2u2q')
    return name.text.strip() if name else None

def get_hotel_rating(soup):
    rating = soup.find('div', class_='css-901oao r-jwli3a r-a5wbuh r-s67bdx r-b88u0q r-10cxs7j r-q4m81j')
    return rating.text.strip() if rating else None

def get_hotel_des(soup):
    des = soup.find('div', attrs={'style': 'font-family:Godwit, -apple-system, BlinkMacSystemFont, Segoe UI, Roboto, Arial, sans-serif, Apple Color Emoji, Segoe UI Emoji, Segoe UI Symbol;font-size:14px;line-height:20px;max-height:80px;overflow:hidden'})
    return des.text.strip().replace('\n', '') if des else None

import re
import json
from geopy.geocoders import Nominatim

def get_hotel_address(soup):
    address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    if address:
        address_full = address.text.strip().replace('\t', '')

        prompt = f"""
            Please separate the following address into 3 parts: street (may have ward), district, city:
            {address_full}

            Ensure that the "district" value is one of the following 12 options: Ba Đình, Cầu Giấy, Đống Đa, Hai Bà Trưng, Hoàn Kiếm, Thanh Xuân, Hoàng Mai, Long Biên, Hà Đông, Tây Hồ, Nam Từ Liêm, Bắc Từ Liêm.
            The "city" value must be Hà Nội.
            For any information not specified in the address return `null`.
            Provide the result in the following JSON format:
            {{
            "street": "...",
            "district": "...",
            "city": "..."
            }}
        """
        response = llm.invoke(prompt)
        response_text = str(response.content)
        try:
            result_dict = json.loads(re.search(r'\{.*?\}', response_text, re.DOTALL).group(0))
            return result_dict
        except (json.JSONDecodeError, AttributeError):
            return {'street': None, 'district': None, 'city': None}

def get_hotel_location(address):
    addr = f"{address['street']}, {address['district']}, {address['city']}"
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(addr)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        return {'latitude': latitude, 'longitude': longitude}
    else:
        return {'latitude': None, 'longitude': None}

def get_hotel_comments(soup):
    comments = []
    comment_divs = soup.findAll('div', class_='css-901oao css-cens5h r-cwxd7f r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')

    for idx, comment in enumerate(comment_divs):
        comments.append({
            'id': idx + 1,  # Optional: adds an ID to each comment
            'text': comment.text.strip()
        })

    return {'comments': comments}

def get_hotel_price(soup):
    room_data = None
    return room_data

def get_hotel_img(soup):
    img_url = None
    return img_url

def extract_hotels_info(url):
    info = {}
    html = get_page_html(url)
    soup = bp(html, 'lxml')
    info['name'] = get_hotel_name(soup)
    info['address'] = get_hotel_address(soup)
    info['location'] = get_hotel_location(info['address'])
    info['rating'] = get_hotel_rating(soup)
    info['description'] = get_hotel_des(soup)
    info['img_url'] = get_hotel_img(soup)
    info['comments'] = get_hotel_comments(soup)
    info['price'] = get_hotel_price(soup)
    return info

def insert_hotel(hotel_info):
    # Database connection details
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    # Convert location data to PostGIS POINT
    location = hotel_info['location']
    point = Point(location['longitude'], location['latitude']) if location['latitude'] and location['longitude'] else None

    # Convert Python objects to JSON strings
    img_url_json = json.dumps(hotel_info['img_url'], ensure_ascii=False)
    comments_json = json.dumps(hotel_info['comments'], ensure_ascii=False)

    # Construct the SQL query
    sql = """
        INSERT INTO Hotel (name, address, location, rating, description, embedding_description, img_url, comments)
        VALUES (%s, ROW(%s, %s, %s), ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s)
    """
    cur.execute(sql, (
        hotel_info['name'],
        hotel_info['address']['street'],
        hotel_info['address']['district'],
        hotel_info['address']['city'],
        point.wkt if point else None,  # Convert point to WKT format
        float(hotel_info['rating']) if hotel_info['rating'] else None,
        hotel_info['description'],
        get_embedding(hotel_info['description']),
        img_url_json,  # Insert the JSON string
        comments_json,  # Insert the JSON string
    ))

    # Commit the transaction and close the connection
    conn.commit()
    cur.close()
    conn.close()

In [64]:
url = "https://www.traveloka.com/vi-vn/hotel/vietnam/hanoi-sunshine-hotel-1000000468475?spec=18-08-2024.19-08-2024.1.1.HOTEL.1000000468475..2"
hotel_info = extract_hotels_info(url)
insert_hotel(hotel_info)

### Restaurant

In [67]:
def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text

def get_restaurant_name(soup):
    name = soup.find('h1', itemprop='name')
    return name.text.strip() if name else None

def get_restaurant_address(soup):
    # Extract address components
    street_address = soup.find('span', itemprop='streetAddress')
    address_locality = soup.find('span', itemprop='addressLocality')
    address_region = soup.find('span', itemprop='addressRegion')

    address_data = {
        'Street': street_address.get_text(strip=True) if street_address else None,
        'District': address_locality.get_text(strip=True) if address_locality else None,
        'City': address_region.get_text(strip=True) if address_region else None
    }
    return address_data

def get_hotel_location(address):
    addr = f"{address['street']}, {address['district']}, {address['city']}"
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(addr)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        return {'latitude': latitude, 'longitude': longitude}
    else:
        return {'latitude': None, 'longitude': None}

def get_restaurant_rating(soup):
    rating = soup.find('div', itemprop='ratingValue', class_='microsite-point-avg')
    return rating.get_text(strip=True) if rating else None

def get_restaurant_description(soup):
    # Extract cuisine type
    cuisine = soup.find('div', itemprop='servesCuisine')
    cuisine_text = cuisine.get_text(strip=True) if cuisine else ""

    # Extract audience
    audience = soup.find('div', class_='audiences')
    audience_text = audience.get_text(strip=True).replace('&nbsp;', ' ') if audience else ""

    # Extract category
    category = soup.find('div', class_='category-items')
    category_text = category.get_text(strip=True) if category else ""

    # Combine all parts into a single description
    description = (
            f"Nhà hàng này thuộc loại {category_text} và phục vụ {cuisine_text}. "
            f"Nó rất phù hợp cho {audience_text}."
        )
    return  description

def get_restaurant_comments(soup):
    comments = []
    comment_elements = soup.findAll('div', class_='comment')
    for comment in comment_elements:
        comments.append(comment.text.strip())
    return comments

def get_restaurant_price_range(soup):
    return json.dumps({"price": "none"})

def get_restaurant_img(soup):
    return None

def extract_restaurant_info(url):
    info = {}
    html = get_page_html(url=url)
    soup = bp(html, 'lxml')
    info['name'] = get_restaurant_name(soup)
    info['address'] = get_restaurant_address(soup)
    info['location'] = get_restaurant_location(info['address'])
    info['rating'] = get_restaurant_rating(soup)
    info['description'] = get_restaurant_description(soup)
    # info['embedding_description '] = get_embedding(info['description'])
    info['price_range'] = get_restaurant_price_range(soup)
    info['img_url'] = get_restaurant_img(soup)
    info['comments'] = get_restaurant_comments(soup)
    return info

def insert_restaurant(restaurant_info):
    # Database connection details
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")

    # Convert location data to PostGIS POINT
    location = restaurant_info['location']
    point = Point(location['longitude'], location['latitude']) if location['latitude'] and location['longitude'] else None

    # Convert Python objects to JSON strings
    comments_json = json.dumps(restaurant_info['comments'], ensure_ascii=False)
    img_url_json = json.dumps(restaurant_info['img_url'], ensure_ascii=False)  # Adjust if needed

    # Construct the SQL query
    sql = """
        INSERT INTO Restaurant (name, address, location, rating, description, price_range, img_url, comments)
        VALUES (%s, ROW(%s, %s, %s), ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s)
    """
    cur.execute(sql, (
        restaurant_info['name'],
        restaurant_info['address']['Street'],
        restaurant_info['address']['District'],
        restaurant_info['address']['City'],
        point.wkt if point else None,
        float(restaurant_info['rating']) if restaurant_info['rating'] else None,
        restaurant_info['description'],
        restaurant_info['price_range'],
        img_url_json,
        comments_json,
    ))

    conn.commit()
    cur.close()
    conn.close()

In [68]:
url = "https://www.foody.vn/ha-noi/kfc-tay-son"
restaurant_info = extract_restaurant_info(url)
insert_restaurant(restaurant_info)